In [1]:
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import pyro
import pyro.distributions as dist

print('Pyro:', pyro.__version__)
print('torch:', torch.__version__)

Pyro: 1.8.4
torch: 1.11.0+cu113


# Bayes' theorem

https://jrnold.github.io/bayesian_notes/bayes-theorem.html

$$P(A \vert B) = \dfrac{P(A, B)}{P(B)} = \dfrac{P(B \vert A) P(A)}{P(B)}$$

https://youtu.be/kqSzLo9fenk


# An Introduction to Bayesian Knowledge Tracing

> Beck, Joseph E., et al. "Does help help? Introducing the Bayesian Evaluation and Assessment methodology." Intelligent Tutoring Systems: 9th International Conference, ITS 2008, Montreal, Canada, June 23-27, 2008 Proceedings 9. Springer Berlin Heidelberg, 2008.
> 
> [link](https://link.springer.com/chapter/10.1007/978-3-540-69132-7_42)

> Corbett, Albert T., and John R. Anderson. "Knowledge tracing: Modeling the acquisition of procedural knowledge." User modeling and user-adapted interaction 4 (1994): 253-278.
>
> [link](http://act-r.psy.cmu.edu/wordpress/wp-content/uploads/2012/12/893CorbettAnderson1995.pdf)

> Yudelson, Michael V., Kenneth R. Koedinger, and Geoffrey J. Gordon. "Individualized bayesian knowledge tracing models." Artificial Intelligence in Education: 16th International Conference, AIED 2013, Memphis, TN, USA, July 9-13, 2013. Proceedings 16. Springer Berlin Heidelberg, 2013.
> 
> [link](https://www.cs.cmu.edu/~ggordon/yudelson-koedinger-gordon-individualized-bayesian-knowledge-tracing.pdf)

> Hawkins, William J., Neil T. Heffernan, and Ryan SJD Baker. "Learning Bayesian knowledge tracing parameters with a knowledge heuristic and empirical probabilities." Intelligent Tutoring Systems: 12th International Conference, ITS 2014, Honolulu, HI, USA, June 5-9, 2014. Proceedings 12. Springer International Publishing, 2014.
>
> [link](https://learninganalytics.upenn.edu/ryanbaker/paper_143.pdf)

Other papers:

* https://arxiv.org/pdf/2105.00385.pdf
* https://educationaldatamining.org/edm2022/proceedings/2022.EDM-long-papers.5/2022.EDM-long-papers.5.pdf

Bayesian Knowledge Tracing is a method to modelding student's knowledges.

* Student knowledge: binary variables (student learns a skill = mastered / not mastered)
* Observations: binary variables (student solves a problem = right / wrong)

## Problem setting

### Model Parameters

* Students: $U$, Skills: $K$, Learned: $L=\lbrace 0, 1\rbrace$, Transit: $T=\lbrace 0, 1\rbrace$, Slip: $S=\lbrace 0, 1\rbrace$, Guess: $G=\lbrace 0, 1\rbrace$

* $p(L_0)$ (or *p-init*): Initial probability of knowing the skill a priori
* $p(T)$ (or *p-transit*): Probability of student's knowledge of a skill transitioning from *not known* to *known* state after an opportunity to apply it
* $p(S)$ (or *p-slip*): Probability to make a mistake when applying a known skill.
* $p(G)$ (or *p-guess*): probability of correctly applying a not-known skill.

The initial probability of student $u$ mastering skill $k$:

$$p(L_1)^k_u = p(L_0)^k$$ 

Whether the student $u$ applied skill $k$(=solve a problem) correctly(1) or incorrectly(0):

$$\begin{aligned}
p(L_{t+1}\vert \text{obs} = 1)^k_u &= \dfrac{p(L_t)^k_u \cdot \big(1 - p(S)^k \big)}{p(L_t)^k_u \cdot \big( 1 - p(S)^k\big) + \big( 1 - p(L_t)^k_u \big) \cdot p(G)^k} \\
p(L_{t+1}\vert \text{obs} = 0)^k_u &= \dfrac{p(L_t)^k_u \cdot p(S)^k}{p(L_t)^k_u \cdot p(S)^k + \big( 1 - p(L_t)^k_u \big) \cdot \big( 1 - p(G)^k \big)}
\end{aligned}$$

* $p(L_t)^k_u \cdot \big(1 - p(S)^k \big)$: Probability that learn the skill doesn't make mistake at $t$.
* $p(L_t)^k_u \cdot p(S)^k$: Probability that learn the skill but make mistake at $t$.
* $\big( 1 - p(L_t)^k_u \big) \cdot p(G)^k$: Probability that doesn't learn the skill but guess correct.
* $\big( 1 - p(L_t)^k_u \big) \cdot \big( 1 - p(G)^k \big)$: Probability that doesn't learn the skill but guess wrong.

The probability of skill mastery: 

$$p(L_{t+1}) = p(L_{t+1}\vert \text{obs}) + \big(1 - p(L_{t+1} \vert obs) \big) \cdot p(T)^k$$

The probability of student $u$ applying the skill $k$ correctly on an upcomming practices opportunity:

$$p(C_{t+1})^k_u = p(L_t)^k_u \cdot \big(1-p(S)^k\big) + \big(1 - p(L_t)^k_u \big) \cdot p(G)^k $$

<img src="./figs/BKT_params.png">

### Solve

1. Evaluation Problem: Given $\lambda = \lbrace \Pi, A, B \rbrace$ and sequence of boservations(practice attempts) $O=\lbrace o_t \rbrace$, $t=\in [1, T]$, calculate the probability that observations are generated given BKT model: $p\lbrace O \vert \lambda \rbrace $
2. Learning Problem: Given BKT paramaters $\lambda$ and a sequence of observations $O$, how should $\lambda$ be adjusted to maximize $p\lbrace O \vert \lambda \rbrace$

In [40]:
# Define data
num_skills = 2  # number of skills
num_students = 5  # number of students
num_obs = 10  # number of observations (exercises)

data = torch.randint(2, size=(num_obs, num_students, num_skills))  # T, U, K
print(data.shape)
print('First student correctness')
print(data[:, 0, :])  # first student


torch.Size([10, 5, 2])
First student correctness
tensor([[1, 0],
        [1, 0],
        [1, 1],
        [0, 0],
        [0, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 1]])


**skip this part**

---

Initialize the probabilities with 4th reference:

$$\begin{aligned} 
p(L_0) &= \dfrac{\sum K_0}{\vert K_0 \vert}\\
P(T) &= \dfrac{\sum_{i \neq 0}(1 - K_{i-1})K_i}{\sum_{i \neq 0} (1 - K_{i-1})} \\
P(G) &= \dfrac{\sum_i C_i (1 - K_i)}{\sum_i (1-K_i)}\\
P(S) &= \dfrac{\sum_i (1 - C_i) K_i}{\sum_i K_i}
\end{aligned}$$

* Here, $K_i$ means the knowledge at problem $i$ (known = 1, unknown = 0), $C_i$ means the correctness at problem $i$ (correct = 1, not correct = 0).

---

In [46]:
# initial probabilities
L_init = torch.tensor(0.5).expand([num_skills])  # K
p_mastery = torch.ones([num_students, num_skills]) * L_init  # U, K
p_transit = torch.tensor(0.5).expand([num_students, num_skills])  # U, K
p_slip = torch.tensor(0.5).expand([num_students, num_skills])  # U, K
p_guess = torch.tensor(0.5).expand([num_students, num_skills])  # U, K

In [49]:
i = 0
mask = data[i] == 1

In [50]:
mask

tensor([[ True, False],
        [ True, False],
        [ True, False],
        [ True,  True],
        [False,  True]])

In [22]:
for i in range(len(data)):
    # L_{t+1} 
    print(data[i])
    mask = data[i] == 1
    data[i][mask]
    break

tensor([[0],
        [0],
        [1],
        [0],
        [1]])


In [25]:
p_mastery[mask] * (1 - )

tensor([0.3333, 0.3333])

In [19]:
data[i] == 1

tensor([[False],
        [False],
        [ True],
        [False],
        [ True]])

In [ ]:
data_init

### Pyro

http://pyro.ai/examples/svi_part_i.html

* latent random variables $z$ $\iff$ `pyro.sample`
* observed random variables $x$ $\iff$ `pyro.sample` with the `obs` keyword argument
* learnable parameters $\theta$ $\iff$ `pyro.param`
* plates $\iff$ `pyro.plate` context managers

In [ ]:
import pyro
import pyro.distributions as dist

pyro.clear_param_store()

In [ ]:

from pyro.nn import PyroModule, PyroParam, PyroSample
from pyro.infer import SVI, Trace_ELBO, Predictive
import torch

In [ ]:
init_prob = pyro.sample('L_init', dist.Beta(1, 1))
learn_prob = pyro.sample('L', dist.Beta(1, 1))
guess_prob = pyro.sample('G', dist.Beta(1, 1))
slip_prob = pyro.sample('S', dist.Beta(1, 1))

p_mastered = torch.ones((num_students, num_skills)) * init_prob

for i in range(len(data)):
    correct_prob = p_mastered * (1 - slip_prob) + (1 - p_mastered) * guess_prob

In [45]:
import pyro
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS
import torch

# Define the BKT model
def bkt_model(data):
    # Priors for initial probability of mastery and transition probabilities
    init_prob = pyro.sample("init_prob", dist.Beta(1, 1))
    learn_prob = pyro.sample("learn_prob", dist.Beta(1, 1))
    guess_prob = pyro.sample("guess_prob", dist.Beta(1, 1))
    slip_prob = pyro.sample("slip_prob", dist.Beta(1, 1))

    # Initial probability of mastery
    p_mastered = init_prob

    # Loop over the student's responses
    for i in range(len(data)):
        # Probability of correct response
        p_correct = p_mastered * (1 - slip_prob) + (1 - p_mastered) * guess_prob

        # Likelihood of response
        pyro.sample(f"obs_{i}", dist.Bernoulli(p_correct).to_event(1), obs=data[i])

        # Update probability of mastery
        p_mastered = p_mastered * (1 - learn_prob) + (1 - p_mastered) * guess_prob

    return p_mastered

# Generate some example data
data = torch.tensor([1, 0, 1, 1, 0, 1, 1])

# Run the model using MCMC inference
kernel = NUTS(bkt_model)
mcmc = MCMC(kernel, num_samples=1000, warmup_steps=100)
mcmc.run(data)

# Extract posterior samples for the initial probability of mastery
init_prob_samples = mcmc.get_samples()["init_prob"]
mean_init_prob = init_prob_samples.mean().item()
std_init_prob = init_prob_samples.std().item()

print(f"Mean initial probability of mastery: {mean_init_prob:.2f}")
print(f"Standard deviation of initial probability of mastery: {std_init_prob:.2f}")


Warmup:   0%|          | 0/1100 [00:00, ?it/s]

ValueError: Expected reinterpreted_batch_ndims <= len(base_distribution.batch_shape), actual 1 vs 0
  Trace Shapes:  
   Param Sites:  
  Sample Sites:  
 init_prob dist |
          value |
learn_prob dist |
          value |
guess_prob dist |
          value |
 slip_prob dist |
          value |

In [37]:
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.infer.autoguide import AutoDelta
from pyro.optim import Adam
from torch.distributions import constraints


# Define model
def bkt(data):
    # Priors
    mastery = pyro.sample("mastery", dist.Beta(1, 1).expand([num_skills]).to_event(1))
    init_know = pyro.sample("init_know", dist.Beta(1, 1).expand([num_students, num_skills]).to_event(1))
    
    # Transition model
    for i in range(num_obs):
        with pyro.plate("data_{}".format(i), num_students):
            # Compute probability of correct response for each student and skill
            p_know = init_know * mastery
            # Update knowledge state based on response
            init_know = pyro.sample("init_know_{}".format(i), 
                                    dist.Beta(p_know * (1 - data[i]) + (1 - p_know) * data[i], 
                                              (1 - p_know) * (1 - data[i]) + p_know * data[i])
                                    .mask(data[i]))
    
    return init_know

# Define guide
guide = AutoDelta(pyro.poutine.block(bkt, expose=['mastery', 'init_know']))

# Run inference using SVI
optim = Adam({'lr': 0.01, 'betas': [0.8, 0.99]})
svi = SVI(bkt, guide, optim, loss=Trace_ELBO())

# Train the model
num_epochs = 1000
for epoch in range(num_epochs):
    loss = svi.step(data)
    if epoch % 100 == 0:
        print("Epoch {}: loss = {}".format(epoch, loss))

# Sample from posterior distribution and apply constraints
posterior_samples = [guide() for _ in range(1000)]
posterior_samples = [torch.clamp(s, min=0, max=1) for s in posterior_samples]


ValueError: Shape mismatch inside plate('data_0') at site init_know_0 dim -1, 5 vs 3
 Trace Shapes:      
  Param Sites:      
 Sample Sites:      
  mastery dist   | 3
         value   | 3
init_know dist 5 | 3
         value 5 | 3
Trace Shapes:
 Param Sites:
Sample Sites: